<a href="https://colab.research.google.com/github/cagBRT/Data/blob/main/Imbalanced_Dataset_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clone the entire repo.
!git clone -s https://github.com/cagBRT/Data.git cloned-repo
%cd cloned-repo

In [ ]:
from pandas import read_csv
from collections import Counter
from matplotlib import pyplot

In [ ]:
filename = '/content/cloned-repo/survivalData.csv'

**Loading the data**<br>

Relevant Information: The dataset contains cases from a study that was conducted between 1958 and 1970 at the University of Chicago's Billings Hospital on the survival of patients who had undergone surgery for breast cancer.<br>

Attribute Information:<br>

Age of patient at time of operation (numerical)<br>

Patient's year of operation (year - 1900, numerical)<br>

Number of positive axillary nodes detected (numerical)<br>

Survival status (class attribute)<br>

>1 = the patient survived 5 years or longer<br>

>2 = the patient died within 5 year<br>

In [ ]:
columns = ['age', 'year', 'nodes', 'class']
dataframe = read_csv(filename, header=None, names=columns)

In [ ]:
dataframe.plot()

In [ ]:
report = dataframe.describe()
print(report)

In [ ]:
dataframe.hist()
pyplot.show()

In [ ]:
target = dataframe['class'].values
counter = Counter(target)
for k,v in counter.items():
  per = v / len(target) * 100
  print('Class=%d, Count=%d, Percentage=%.3f%%' % (k, v, per))